In [2]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optiim
from torchvision import datasets, transforms

import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [5]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("../data", train = True, download = True,
                  transform = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))
                  ])),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("../data", train = False,
                  transform = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size = test_batch_size,
    shuffle = True
)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # Feature Extraction
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        
        # print(x.shape)
        # Fully Connected(Classification)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

## Optimization ##
- Model과 Optimization 설정

In [9]:
model = Net().to(device)

In [10]:
optimizer = optiim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

- Parametres들 확인
 - Weight, Bais를 순서대로 보여줌

In [14]:
params = list(model.parameters())
for i in range(8):
    print(params[i].shape)

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


## Before Training ##
- 학습하기 전에 Model이 Train할수 있도록 Train Mode로 변환
 - Convolution 또는 Linear 뿐만 아니라, Dropout과 추후에 배우게 될 Batch Nomalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [15]:
model.train() # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch  데이터추출

In [16]:
data, target = next(iter(train_loader))

In [17]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [19]:
data, target = data.to(device), target.to(device)

In [20]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [21]:
# optimizer Clear
optimizer.zero_grad()

- 준비한 데이터를 model에 input으로 넣어 output을 얻음

In [22]:
output = model(data)

- MOdel에서 예측한 결과를 Loss Function에 넣음
 - 여기 예제에서는 Negative Log-Likelihood Loss라는 Loss Function을 사용

In [23]:
loss = F.nll_loss(output, target)

- Back Propagtaion을 통해 Gradients를 계산

In [24]:
loss.backward()

- 계산된 Gradients는 계산된 걸로 끝이 아니라 Parameter에 Update

In [25]:
optimizer.step()

## Start Training ##

위의 최적화 과정을 반복하여 학습 시작

In [26]:
epochs = 1
log_interval = 100

In [46]:
for epoch in range(1, epochs + 1):
    # Train mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{} / {} <{:.0f}%>]\tLoss: {:.6f}".format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100*batch_idx / len(train_loader), loss.item()))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction = "sum").item()
            pred = output.argmax(dim = 1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print("\nTest set: Avearge Loss {:.4f}, Accuracy: {}/{} <{:.0f}%>\n".format(
        test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)
    ))

Train Epoch: 1 [0 / 60000 <0%>]	Loss: 0.090405
Train Epoch: 1 [6400 / 60000 <11%>]	Loss: 0.087489
Train Epoch: 1 [12800 / 60000 <21%>]	Loss: 0.123433
Train Epoch: 1 [19200 / 60000 <32%>]	Loss: 0.098978
Train Epoch: 1 [25600 / 60000 <43%>]	Loss: 0.037894
Train Epoch: 1 [32000 / 60000 <53%>]	Loss: 0.070124
Train Epoch: 1 [38400 / 60000 <64%>]	Loss: 0.035003
Train Epoch: 1 [44800 / 60000 <75%>]	Loss: 0.029384
Train Epoch: 1 [51200 / 60000 <85%>]	Loss: 0.054256
Train Epoch: 1 [57600 / 60000 <96%>]	Loss: 0.063137

Test set: Avearge Loss 0.0589, Accuracy: 9814/10000 <98%>



## Evaluation ##
- 앞에서 model.train() 모드로 변환 것처럼 평가 할때는 eval()로 설정
 - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [28]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engin, 즉 backporpagation이나 gradient 계산 등ㄹ을 꺼서 memory usage를 줄이고 속도를 높임

In [ ]:
test_loss = 0
correct = 0

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction="sum").item()
    
    # keepdim : 차원수 유지
    pred  = output.argmax(dim = 1, keepdim = True)
    correct = pred.eq(target.view_as(pred)).sum().item()

In [34]:
output.shape

torch.Size([64, 10])

In [35]:
pred.shape

torch.Size([64, 1])

In [39]:
target.view_as(pred).shape

torch.Size([64, 1])

In [40]:
pred.eq(target.view_as(pred)).sum().item() / 64

0.953125

In [30]:
test_loss

7.020183563232422

In [31]:
correct

61

In [41]:
test_loss /= len(test_loader.dataset)

In [42]:
test_loss

0.0007020183563232422

In [45]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction = "sum").item()
        pred = output.argmax(dim = 1, keepdim = True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

print("\nTest set: Avearge Loss {:.4f}, Accuracy: {}/{} <{:.0f}%>\n".format(
    test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)
))


Test set: Avearge Loss 0.1190, Accuracy: 9627/10000 <96%>

